# Tweepy for twitter API 2.0

"An easy-to-use Python library for accessing the Twitter API"

由於如果要打撈Twitter資料就必須去看懂Twitter API，有些熱心的程式設計師可能會為了自己方便並與別人分享，而設計一個第三方的函式庫，裡面有一些算好用的函式可以避免讀者必須查閱Twitter API才會知道有哪些欄位或哪些資料可以取用。而Tweepy就是Twitter API的第三方client-side函式庫。不是Twitter官方提供的，而是熱心的一群人所提供的。

但好不好用見仁見智。若以「好用」的觀點來看的話，tweepy有將回傳資料的推特文和使用者打包為物件導向的資料，甚至也已經把字串型態的時間戳記給轉為Python的Datetime物件，也是可以避免掉使用者的一些麻煩的。若以「不好用」的觀點來看的話，如果很熟悉Twitter API的話，那有哪些欄位可以取用都是很清楚的，而使用Tweepy一樣要看懂有哪些欄位可以取用，反而是如果Twitter API一改，而tweepy還沒改的話，就有一些功能不能用，偏偏，這種社群網站的API為了隱私或倫理的問題，是經常更動的。而且，因為被打包為物件，所以必須要懂得tweepy把這些資料打包為什麼樣的物件，反而還要多弄懂tweepy怎麼存放資料。因此，有些人是寧可用原生Twitter API來爬取資料的。

## API 2.0 with tweepy

以下是搭配tweepy for Twitter API 2.0的函式操作結果
* `wait_on_rate_limit`: 由於Twitter對於每個App在一段時間內（15分鐘、1個月內）能取用多少資料（[rate limit](https://developer.twitter.com/en/docs/twitter-api/rate-limits)），當爬取速度高於速度限制時，程式往往就會終止。但在tweepy在初始化`Client`時提供`wait_on_rate_limit`的選項，讓程式在超過速度限制而被Twitter拒絕回傳資料時，可以自動等候，而不會跳開程式執行。
* `return_type`: tweepy所抓回來的結果預設為tweepy所自己設計的Response型態，但也可以透過設定`return_type = dict`要求將回傳的資料以Dictionary型態來表示。就看程式寫作者習慣用哪一種型態。如果傳回預設的Tweepy Response型態，因為Twitter API傳回來的資料有時間，就會被自動轉為Datetime物件；如果要求要傳回Dictionary型態，那自然不會自動幫忙把字面上的時間轉為Datetime型態，而是以字串的型態來紀錄。

```python
client = tweepy.Client(bearer_token=bearer_token, 
                       consumer_key=consumer_key, 
                       consumer_secret=consumer_secret, 
                       access_token=access_token, 
                       access_token_secret=access_token_secret, 
                       # default:Response
                       return_type = requests.Response, 
                       # return_type = dict,
                       wait_on_rate_limit=True)
```                       

### Insitaize a client

```python
import tweepy

# token for essentials
bearer_token = 'YOUR TOKEN'

# client = tweepy.Client(bearer_token, wait_on_rate_limit=True, return_type=dict)
client = tweepy.Client(bearer_token, 
                       # return_type = dict,
                       wait_on_rate_limit=True)
api = tweepy.API(client)
```

### Query: Recent search

* https://docs.tweepy.org/en/stable/client.html#tweepy.Client.search_all_tweets
* https://towardsdatascience.com/an-extensive-guide-to-collecting-tweets-from-twitter-api-v2-for-academic-research-using-python-3-518fcb71df2a
* Very Useful! https://dev.to/twitterdev/a-comprehensive-guide-for-using-the-twitter-api-v2-using-tweepy-in-python-15d9


Queries:
* `query = 'from:BarackObama -is:retweet'`

Fields: 
* https://developer.twitter.com/en/docs/twitter-api/fields

```python
tweets = client.search_recent_tweets("#pelosi is:retweet", 
                                  tweet_fields =['id','text','author_id','in_reply_to_user_id',
                                                 'geo','conversation_id','created_at','lang',
                                                 'public_metrics','referenced_tweets','source'],
                                  user_fields=['name','username','created_at','public_metrics'],
                                  expansions='author_id',
                                  max_results=10)
```

```python
tweets.data
```

```python
print(tweets.meta['next_token'])
print(tweets.meta['newest_id'])
print(tweets.meta['oldest_id'])
tweets.includes['users']
```

```python
# print(tweets.data[1].text)
tweets.data[1].created_at
tweets.data[1].public_metrics
dict(tweets.data[1])
```

```python
# Initialization

all_tweets = []
all_users = []
tweets = client.search_recent_tweets("#pelosi is:retweet", 
                                  tweet_fields =['id','text','author_id','in_reply_to_user_id',
                                                 'geo','conversation_id','created_at','lang',
                                                 'public_metrics','referenced_tweets','source'],
                                  user_fields=['name','username','created_at','public_metrics','profile_image_url'],
                                  expansions='author_id',
                                  max_results=10)
all_tweets = tweets.data
all_users = tweets.includes['users']
```

```python

for i in range(10):
    tweets = client.search_recent_tweets("#pelosi is:retweet", 
                                         tweet_fields =['id','text','author_id','in_reply_to_user_id',
                                                        'geo','conversation_id','created_at','lang',
                                                        'public_metrics','referenced_tweets','source'],
                                         user_fields=['name','username','created_at','public_metrics','profile_image_url'],
                                         expansions='author_id',
                                         next_token=tweets.meta["next_token"],
                                         max_results=10)
    all_tweets.extend(tweets.data)
    all_users.extend(tweets.includes['users'])
    print("All(n={:4}) {}\t{}".format(len(all_tweets), tweets.meta["newest_id"], tweets.meta["oldest_id"])) 
```

## Expansions

### Get user info

```python
for user in all_users[:10]:
    print(user.username, user.profile_image_url)
```    

### Get media

```python
import tweepy

# Replace with your own search query
query = 'covid -is:retweet has:media'

tweets = client.search_recent_tweets(query=query, tweet_fields=['context_annotations', 'created_at'],
                                     media_fields=['preview_image_url'], expansions='attachments.media_keys',
                                     max_results=10)
tweets.includes['media']
```

```python
for tweet in tweets.data:
    print(tweet.text)
    if len(tweet.context_annotations) > 0:
        print(tweet.context_annotations)
```